# Libraries Import

## Regular Libraries

In [1]:
import numpy as np
import pandas as pd
import os

## Auxiliary Library

In [2]:
from glob import glob

from tqdm import tqdm

import random

## Data Visualization Library

In [3]:
import matplotlib.pyplot as plt

In [4]:
from transformers import RobertaTokenizerFast, RobertaForTokenClassification
from torch.utils.data import Dataset, DataLoader
import pdb
import torch
from torch import cuda
from sklearn.metrics import accuracy_score

MODEL_NAME = "../input/roberta-base"

---

# Data Extraction

## Main Data

In [5]:
train = pd.read_csv('../input/feedback-prize-2021/train.csv')
sample_submission = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')

In [6]:
print(train.shape)
train.head()

(144293, 8)


,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [7]:
print(f'Training data consists of {train.shape[0]} annotaions')

Training data consists of 144293 annotaions


In [8]:
print(train["predictionstring"].nunique())
print("------------------------------------")
print(train["discourse_type_num"].nunique())
print(train["discourse_type_num"].unique())
print("------------------------------------")
print(train["discourse_type"].unique())

50355
------------------------------------
44
['Lead 1' 'Position 1' 'Evidence 1' 'Evidence 2' 'Claim 1' 'Evidence 3'
 'Evidence 4' 'Claim 2' 'Evidence 5' 'Concluding Statement 1'
 'Counterclaim 1' 'Rebuttal 1' 'Claim 3' 'Claim 4' 'Claim 5' 'Claim 6'
 'Claim 7' 'Counterclaim 2' 'Rebuttal 2' 'Counterclaim 3' 'Rebuttal 3'
 'Evidence 6' 'Lead 2' 'Counterclaim 4' 'Counterclaim 5' 'Counterclaim 6'
 'Evidence 7' 'Claim 8' 'Evidence 8' 'Concluding Statement 2' 'Rebuttal 4'
 'Rebuttal 5' 'Claim 9' 'Position 2' 'Claim 10' 'Claim 11' 'Claim 12'
 'Evidence 9' 'Concluding Statement 3' 'Concluding Statement 4'
 'Evidence 10' 'Evidence 11' 'Rebuttal 6' 'Evidence 12']
------------------------------------
['Lead' 'Position' 'Evidence' 'Claim' 'Concluding Statement'
 'Counterclaim' 'Rebuttal']


In [9]:
train["discourse_len"] = train["discourse_text"].apply(lambda x:len(x.split()))

In [10]:
train["predictionstring_len"] = train["predictionstring"].apply(lambda x:len(x.split()))

In [11]:
train["len_predict"] = 0

In [12]:
for i in range(0,len(train["discourse_len"])):
    if train["discourse_len"][i] == train["predictionstring_len"][i]:
        train["len_predict"][i] = 0
    else:
        train["len_predict"][i] = 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [13]:
train["len_predict"].sum()

468

In [14]:
train.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,discourse_len,predictionstring_len,len_predict
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,44,44,0
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,15,15,0
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,16,16,0
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,63,63,0
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,24,24,0


In [15]:
#test_names, test_texts = [], []
#for f in tqdm(list(os.listdir('../input/feedback-prize-2021/test'))):
#    test_names.append(f.replace('.txt', ''))
#    test_texts.append(open('../input/feedback-prize-2021/test/' + f, 'r').read())
#test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
#
#test_texts.head()

In [16]:
test_names, train_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/train'))):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r').read())
train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})

train_text_df.head()

100%|██████████| 15594/15594 [01:05<00:00, 238.29it/s]


,id,text
0,62C57C524CD2,I think we should be able to play in a sport i...
1,80667AD3FFD8,Some schools require summer projects for stude...
2,21868C40B94F,Driverless cars have been argued and talked ab...
3,87A6EF3113C6,"The author of ""The Challenge of Exploring Venu..."
4,24687D08CFDA,"Wow, from the mar really look like humans face..."


In [17]:
train_df = train.copy()
all_entities = []
for i in tqdm(train_text_df.iterrows()):
    total = i[1]['text'].split(' ').__len__()
    start = -1
    entities = []
    for j in train_df[train_df['id'] == i[1]['id']].iterrows():
        discourse = j[1]['discourse_type']
        list_ix = j[1]['predictionstring'].split(' ')
        ent = [f"I-{discourse}" for ix in list_ix]
        ent[0] = f"B-{discourse}"
        ds = int(list_ix[0])
        de = int(list_ix[-1])
        if start < ds-1:
            ent_add = ['O' for ix in range(int(ds-1-start))]
            ent = ent_add + ent
        entities.extend(ent)
        start = de
    if len(entities) < total:
        ent_add = ["O" for ix in range(total-len(entities))]
        entities += ent_add
    else:
        entities = entities[:total]
    all_entities.append(entities)

15594it [05:43, 45.40it/s]


In [18]:
train_text_df['entities'] = all_entities
train_text_df.head()

,id,text,entities
0,62C57C524CD2,I think we should be able to play in a sport i...,"[B-Position, I-Position, I-Position, I-Positio..."
1,80667AD3FFD8,Some schools require summer projects for stude...,"[B-Position, I-Position, I-Position, I-Positio..."
2,21868C40B94F,Driverless cars have been argued and talked ab...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
3,87A6EF3113C6,"The author of ""The Challenge of Exploring Venu...","[B-Position, I-Position, I-Position, I-Positio..."
4,24687D08CFDA,"Wow, from the mar really look like humans face...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."


In [19]:
config = {'model_name': '/kaggle/input/roberta-base/',
         'max_length': 512,
         'train_batch_size':8,
         'valid_batch_size':16,
         'epochs':3,
         'learning_rate':1e-05,
         'max_grad_norm':10,
         'device': 'cuda' if cuda.is_available() else 'cpu'}

In [20]:
output_labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [21]:
print(labels_to_ids)
print(ids_to_labels)

{'O': 0, 'B-Lead': 1, 'I-Lead': 2, 'B-Position': 3, 'I-Position': 4, 'B-Claim': 5, 'I-Claim': 6, 'B-Counterclaim': 7, 'I-Counterclaim': 8, 'B-Rebuttal': 9, 'I-Rebuttal': 10, 'B-Evidence': 11, 'I-Evidence': 12, 'B-Concluding Statement': 13, 'I-Concluding Statement': 14}
{0: 'O', 1: 'B-Lead', 2: 'I-Lead', 3: 'B-Position', 4: 'I-Position', 5: 'B-Claim', 6: 'I-Claim', 7: 'B-Counterclaim', 8: 'I-Counterclaim', 9: 'B-Rebuttal', 10: 'I-Rebuttal', 11: 'B-Evidence', 12: 'I-Evidence', 13: 'B-Concluding Statement', 14: 'I-Concluding Statement'}


In [22]:
tokenizer = RobertaTokenizerFast.from_pretrained(MODEL_NAME)

model = RobertaForTokenClassification.from_pretrained(config['model_name'], num_labels=len(output_labels))

Some weights of the model checkpoint at /kaggle/input/roberta-base/ were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at /kaggle/input/roberta-base/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on

In [23]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.text[index]
        word_labels = self.data.entities[index]

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
#                              is_pretokenized=True, 
#                                   is_split_into_words=True,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True,
                            max_length=self.max_len)
        
        # step 3: create token labels only for first word pieces of each tokenized word
#         pdb.set_trace()
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
#         print(len(sentence), len(labels))
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
#             print(idx)
            if mapping[0] != 0 and mapping[0] != encoding['offset_mapping'][idx-1][1]:
            # overwrite label
#             pdb.set_trace()
#             print(mapping)
#             print(encoded_labels.shape, len(labels), idx, i)
                try:
                    encoded_labels[idx] = labels[i]
                except:
                    pass
                i += 1
            else:
                if idx==1:
    #                 print(idx)
                    encoded_labels[idx] = labels[i]
                    i += 1
        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

    def __len__(self):
        return self.len
                                  

In [24]:
data = train_text_df[['text', 'entities']]
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, config['max_length'])
testing_set = dataset(test_dataset, tokenizer, config['max_length'])

FULL Dataset: (15594, 2)
TRAIN Dataset: (12475, 2)
TEST Dataset: (3119, 2)


In [25]:
train_params = {'batch_size': config['train_batch_size'],
                'shuffle': True,
                'num_workers': 1,
                'pin_memory':True
                }

test_params = {'batch_size': config['valid_batch_size'],
                'shuffle': True,
                'num_workers': 1,
                'pin_memory':True
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [26]:
device = config['device']
device

'cuda'

In [27]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [28]:
model.to(device)
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels,
               return_dict=False)
initial_loss = outputs[0]
initial_loss

tensor(2.9545, device='cuda:0', grad_fn=<NllLossBackward>)

In [29]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=config['learning_rate'])

In [30]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels,
                               return_dict=False)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)
        
        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=config['max_grad_norm']
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [31]:
for epoch in range(config['epochs']):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Training loss per 100 training steps: 2.918163299560547
Training loss per 100 training steps: 1.5868904873876288
Training loss per 100 training steps: 1.3530007648230784
Training loss per 100 training steps: 1.2328100917347247
Training loss per 100 training steps: 1.157837126320437
Training loss per 100 training steps: 1.1064888686239125
Training loss per 100 training steps: 1.0693214369097883
Training loss per 100 training steps: 1.0380558656387764
Training loss per 100 training steps: 1.0143122671546412
Training loss per 100 training steps: 0.9943706076364803
Training loss per 100 training steps: 0.9728972908857462
Training loss per 100 training st

In [32]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels,
                                     return_dict=False)
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [33]:
labels, predictions = valid(model, testing_loader)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Validation loss per 100 evaluation steps: 0.3423628509044647
Validation loss per 100 evaluation steps: 0.33017254716689043
Validation Loss: 0.33508592973916956
Validation Accuracy: 0.38759424866601255


In [34]:
sentence = "@HuggingFace is a company based in New York, but is also has employees working in Paris"
model.eval()
def inference(sentence):
    inputs = tokenizer(sentence,
#                         is_split_into_words=True, 
                        return_offsets_mapping=True, 
                        padding='max_length', 
                        truncation=True, 
                        max_length=config['max_length'],
                        return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = model(ids, attention_mask=mask, return_dict=False)
    logits = outputs[0]
    
    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    out_str = []
    off_list = inputs["offset_mapping"].squeeze().tolist()
    for idx, mapping in enumerate(off_list):
#         print(mapping, token_pred[1], token_pred[0],"####")

#         only predictions on first word pieces are important
        if mapping[0] != 0 and mapping[0] != off_list[idx-1][1]:
#             print(mapping, token_pred[1], token_pred[0])
            prediction.append(wp_preds[idx][1])
            out_str.append(wp_preds[idx][0])
        else:
            if idx == 1:
                prediction.append(wp_preds[idx][1])
                out_str.append(wp_preds[idx][0])
            continue
    return prediction, out_str

In [35]:
test_names, test_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/test'))):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('../input/feedback-prize-2021/test/' + f, 'r').read())
test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
# test_texts['text'] = test_texts['text'].apply(lambda x:x.split())
test_texts.head()

100%|██████████| 5/5 [00:00<00:00, 296.42it/s]


,id,text
0,0FB0700DAF44,"During a group project, have you ever asked a ..."
1,D72CB1C11673,Making choices in life can be very difficult. ...
2,18409261F5C2,80% of Americans believe seeking multiple opin...
3,DF920E0A7337,Have you ever asked more than one person for h...
4,D46BCB48440A,"When people ask for advice,they sometimes talk..."


In [36]:
# test_texts = train_text_df['text'].tolist()[:10]
y_pred = []

for i, t in enumerate(test_texts['text'].tolist()):
    o,o_t = inference(t)
    y_pred.append(o)
    l = train_text_df['entities'][i]

In [37]:
final_preds = []
import pdb
for i in tqdm(range(len(test_texts))):
#     pdb.set_trace()
    idx = test_texts.id.values[i]
#     pred = ['']*len(test_texts[i])

#     for j in range(len(y_pred[i])):
#         if words[i][j] != None:
#             pred[words[i][j]] = labels[y_pred[i][j]]

    pred = [x.replace('B-','').replace('I-','') for x in y_pred[i]]
#     print(pred)
    preds = []
    j = 0
    while j < len(pred):
        cls = pred[j]
#         pdb.set_trace()
        if cls == 'O':
            j += 1
        end = j + 1
        while end < len(pred) and pred[end] == cls:
            end += 1
            
        if cls != 'O' and cls != '' and end - j > 10:
            final_preds.append((idx, cls, ' '.join(map(str, list(range(j, end))))))
        
        j = end
        
print(final_preds[1])

100%|██████████| 5/5 [00:00<00:00, 2415.52it/s]

('0FB0700DAF44', 'Claim', '48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107')


In [38]:
print(len(final_preds))
test_df = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')
test_df

30


,id,class,predictionstring
0,18409261F5C2,NaN,NaN
1,D46BCB48440A,NaN,NaN
2,0FB0700DAF44,NaN,NaN
3,D72CB1C11673,NaN,NaN
4,DF920E0A7337,NaN,NaN


In [39]:
sub = pd.DataFrame(final_preds)
sub.columns = test_df.columns

In [40]:
sub.head()

,id,class,predictionstring
0,0FB0700DAF44,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,0FB0700DAF44,Claim,48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 6...
2,0FB0700DAF44,Position,108 109 110 111 112 113 114 115 116 117 118 119
3,0FB0700DAF44,Claim,122 123 124 125 126 127 128 129 130 131 132 13...
4,0FB0700DAF44,Evidence,147 148 149 150 151 152 153 154 155 156 157 15...


In [41]:
sub.to_csv("submission.csv", index=False)